# IMPORT LIBRARY

In [4]:
# Data Manipulation and Graph
import pandas as pd
import matplotlib.pyplot as plt

# Modelling
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor

# Evaluation
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error, r2_score

In [6]:
df= pd.read_csv('Regresi A5_final.csv')
df.head(10)

,year,month,day,stasiun,pm10,pm25,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida
0,2024,1,1,DKI5 Kebon Jeruk Jakarta Barat,58.0,99.0,40.0,18.0,55.0,12.0
1,2024,1,2,DKI5 Kebon Jeruk Jakarta Barat,40.0,68.0,40.0,9.0,65.0,11.0
2,2024,1,3,DKI5 Kebon Jeruk Jakarta Barat,32.0,54.0,39.0,7.0,63.0,10.0
3,2024,1,4,DKI5 Kebon Jeruk Jakarta Barat,38.0,59.0,46.0,10.0,55.0,12.0
4,2024,1,5,DKI5 Kebon Jeruk Jakarta Barat,30.0,52.0,37.0,15.0,48.0,15.0
5,2024,1,6,DKI5 Kebon Jeruk Jakarta Barat,32.0,64.0,39.0,12.0,61.0,14.0
6,2024,1,7,DKI5 Kebon Jeruk Jakarta Barat,53.0,86.0,40.0,23.0,62.0,14.0
7,2024,1,8,DKI5 Kebon Jeruk Jakarta Barat,37.0,55.0,38.0,12.0,43.0,14.0
8,2024,1,9,DKI5 Kebon Jeruk Jakarta Barat,54.0,81.0,21.0,18.0,35.0,3.0
9,2024,1,10,DKI5 Kebon Jeruk Jakarta Barat,43.0,73.0,18.0,11.0,70.0,7.0


# DATA SPLITTING

## Scenario 1: 70/30

In [8]:
df_train70= pd.read_csv("csv/data_train_0.7.csv")
df_test70= pd.read_csv("csv/data_test_0.7.csv")

In [13]:
df_train70.head()

,year,month,day,stasiun,pm10,pm25,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida
0,2024,1,1,DKI5 Kebon Jeruk Jakarta Barat,58.0,99.0,40.0,18.0,55.0,12.0
1,2024,1,2,DKI5 Kebon Jeruk Jakarta Barat,40.0,68.0,40.0,9.0,65.0,11.0
2,2024,1,3,DKI5 Kebon Jeruk Jakarta Barat,32.0,54.0,39.0,7.0,63.0,10.0
3,2024,1,4,DKI5 Kebon Jeruk Jakarta Barat,38.0,59.0,46.0,10.0,55.0,12.0
4,2024,1,5,DKI5 Kebon Jeruk Jakarta Barat,30.0,52.0,37.0,15.0,48.0,15.0


In [14]:
df_test70.head()

,year,month,day,stasiun,pm10,pm25,sulfur_dioksida,karbon_monoksida,ozon,nitrogen_dioksida
0,2025,1,17,DKI5 Kebon Jeruk Jakarta Barat,11.0,54.0,26.0,4.0,50.0,5.0
1,2025,1,18,DKI5 Kebon Jeruk Jakarta Barat,7.0,25.0,11.0,3.0,55.0,5.0
2,2025,1,19,DKI5 Kebon Jeruk Jakarta Barat,11.0,37.0,24.0,3.0,49.0,3.0
3,2025,1,20,DKI5 Kebon Jeruk Jakarta Barat,10.0,43.0,18.0,3.0,52.0,5.0
4,2025,1,21,DKI5 Kebon Jeruk Jakarta Barat,15.0,54.0,3.0,12.0,38.0,13.0


## Scenario 2: 80/20

In [12]:
df_train80= pd.read_csv('csv/data_train_0.8.csv')
df_test80= pd.read_csv('csv/data_test_0.8.csv')

# MODEL TRAINING

## Linear Regression

### S1: 70/30

### S2: 80/20

## Gradient Boosting

### S1: 70/30

### S2: 80/20

## ADA Boost

### S1: 70/30

### S2: 80/20

## XG Boost

### S1: 70/30

### S2: 80/20

# MODEL EVALUATION

## MAE

## RMSE

## MAPE

## R-squared

# Export Requirements.txt

In [3]:
import pkg_resources

packages = [
    'pandas',
    'scikit-learn',
]

with open("lukas_requirements.txt", "w") as f:
    for pkg in packages:
        try:
            version = pkg_resources.get_distribution(pkg).version
            f.write(f"{pkg}=={version}\n")
        except pkg_resources.DistributionNotFound:
            print(f"Package {pkg} not found in the environment.")
